In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint
from pathlib import Path
from random import randint
import json

import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import bids
from bids import BIDSLayout
from scipy.ndimage import zoom, binary_dilation
import h5py
import nibabel as nib
from einops import rearrange

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from tc2see import load_data

from sklearn.model_selection import KFold
from fracridge import FracRidgeRegressorCV
from metrics import (
    cosine_distance, squared_euclidean_distance, r2_score, two_versus_two,
    two_versus_two_slow
)
import warnings

from noise_ceiling import (
    compute_ncsnr,
    compute_nc,
)

In [2]:
dataset_root = Path('E:\\fmri_processing\\results')
tc2see_version = 3
dataset_path = dataset_root
derivatives_path = dataset_path / 'derivatives_TC2See'
data_path = derivatives_path / 'fmriprep'

In [3]:
ROIs = {
    "FFC": [18],"V1": [1],"V2": [4],"V3": [5],"V3A": [13],"V3B": [19],"V3CD": [158],"V4": [6],"V6": [3],"V7": [16],
    "V8": [7], "VMV1": [153],"VMV2": [160],"VMV3": [154],"LO1": [20],"LO2": [21],"PIT": [22],"VVC": [163], "140": [140], "11":[11],
    "85": [85], "83":[83], "82": [82], "87": [87]
}

ROI_combos = {
    "all_rois": [ roi for roi in range(1,181)],
    "all_but_FFC": [ roi for roi in range(1,181) if roi != 18],
    "all_but_FFC_VVC": [ roi for roi in range(1,181) if roi not in [18, 163]],
    "V1_to_V8": [1, 4, 5, 6, 3, 16, 7],
    "visual_but_not_V1_to_V8": [18, 13, 19, 158, 153, 160, 154, 20, 21, 22, 163],
    "visual_but_not_FFC_VVC": [1, 4, 5, 6, 3, 16, 7, 13, 19, 158, 153, 160, 154, 20, 21, 22],
    "visual_but_not_V1_to_V8_or_FFC_VVC": [ 13, 19, 158, 153, 160, 154, 20, 21, 22],
    "PH_and_PEF": [140, 11, 85, 83, 82, 87],
    "PH_and_PEF_FFC_VVC": [140, 11, 85, 83, 82, 87, 18, 163]
}

In [4]:
glasser_L = nib.freesurfer.io.read_annot("E:/fmri_processing/results/visualization/atlas/lh.HCPMMP1.annot")
glasser_R = nib.freesurfer.io.read_annot("E:/fmri_processing/results/visualization/atlas/rh.HCPMMP1.annot")

In [5]:
ROI_masks = {}

for key, vals in ROIs.items():

    # mask glasser atlas to mark current loop ROI as 1s
    L_mask = np.isin(glasser_L[0], vals) # vals is a list of ROIs to set as 1
    R_mask = np.isin(glasser_R[0], vals)
    print(f"{key}: {sum(L_mask) + sum(R_mask)}")
    
    # concatenate left and right hemispheres 
    L_R_concat_mask = np.concatenate([L_mask, R_mask], axis=0)
    ROI_masks[key] = L_R_concat_mask

FFC: 1670
V1: 8359
V2: 6043
V3: 3909
V3A: 1403
V3B: 734
V3CD: 768
V4: 2858
V6: 1077
V7: 560
V8: 653
VMV1: 877
VMV2: 687
VMV3: 662
LO1: 586
LO2: 543
PIT: 678
VVC: 1241
140: 1424
11: 855
85: 1267
83: 1805
82: 1418
87: 1516


In [6]:
model_name = 'ViT-B=32'
embedding_name = 'embedding' 
num_runs = 6
tr = 2 # 1.97

with h5py.File(derivatives_path / f'{model_name}-features.hdf5', 'r') as f:
    stimulus = f[embedding_name][:]

load_data_params = dict(
    path = data_path / f'tc2see-v{tc2see_version}-fsaverage-surfs.hdf5', 
    tr_offset = num_runs / tr,
    run_normalize='linear_trend',
    interpolation=False,
)

In [7]:
accuracies = {}
subjs = [str(sub) if sub >= 10 else '0'+str(sub) for sub in range(5,35)] 
top_n_nc_vals = 256

for subj in tqdm(subjs):
    print(f"====== Subject {subj} ======")
    accuracies[subj] = {}
    try:
        subject = f'sub-{subj}'

        # stimulus_ids are needed for the two_versus_two accuracy scores below
        _, stimulus_ids = load_data(
            data_path / f'tc2see-v{tc2see_version}-fsaverage-surfs.hdf5', 
            subject,
            tr_offset=6 / tr,
            run_normalize='linear_trend',
            interpolation=False,
        )

        accuracies_list = []
        std_list = []

        for ROI, ROI_mask in ROI_masks.items():
            print(f"- Decoding {ROI}...")
            accuracies[subj][ROI] = {}
            ROI_accuracies_list = []
            
            # Cross validation. Use every run as test data once.
            for test_run_id in range(num_runs):
                training_run_ids = list(range(num_runs))
                training_run_ids.remove(test_run_id) # Remove the test data id 

                # load the training and test data
                bold_train, stimulus_ids_train = load_data(
                    **load_data_params,
                    subject = subject,
                    run_ids = training_run_ids
                )

                bold_test, stimulus_ids_test = load_data(
                    **load_data_params,
                    subject = subject,
                    run_ids = [test_run_id]
                )

                ncsnr = compute_ncsnr(bold_train, stimulus_ids_train) # Compute noise ceiling noise ratio
                nc = compute_nc(ncsnr, num_averages=1)

                nc[~ROI_mask] = 0
                argsort_ids = np.argsort(-nc) # Default ascending, make descending 


                if np.count_nonzero(nc) < top_n_nc_vals:
                    argsort_ids = argsort_ids[:np.count_nonzero(nc)] 
                    if test_run_id == 0: print(f"   Using top {np.count_nonzero(nc)} nc values")
                else:
                    argsort_ids = argsort_ids[:top_n_nc_vals] 
                    if test_run_id == 0: print(f"   Using top {top_n_nc_vals} nc values")
                     
                X_train = bold_train[:, argsort_ids] 

                X_nan_train = np.isnan(X_train) # Checks if any not a number values in x and sets those to zero
                X_train[X_nan_train] = 0.

                X_test = bold_test[:, argsort_ids]
                X_nan_test = np.isnan(X_test) # Checks if any not a number values in x and sets those to zero
                X_test[X_nan_test] = 0.
 
                Y_train = stimulus[stimulus_ids_train] 
                Y_test = stimulus[stimulus_ids_test]

                model = FracRidgeRegressorCV()
                model.fit(X_train, Y_train)
                Y_test_pred = model.predict(X_test) # Y_test and Y_test_pred are n x 512 matrics (n is the number of birds).

                distances = cosine_distance(
                    torch.from_numpy(Y_test[None]).float(), 
                    torch.from_numpy(Y_test_pred[:, None]).float()
                ) # Y_test(1, N, 512) & Y_test_pred(N, 1, 512) converted to pytorch arrays from np

                accuracy = round(two_versus_two(distances, stimulus_ids=stimulus_ids).item() * 100, 2) 
                ROI_accuracies_list.append(accuracy)
            
            # get average of accuracies for this iteration of runs
            ROI_accuracy_avg = np.mean(ROI_accuracies_list)
            accuracies[subj][ROI] = round(ROI_accuracy_avg, 3)
            print(f"   Accuracy: {round(ROI_accuracy_avg, 3)}")

            # get standard deviation of accuracies for this iteration of runs
            ROI_accuracy_std = np.std(ROI_accuracies_list)
            print(f"   Std: {round(ROI_accuracy_std, 3)}")

            # save accuracies checkpoint
            with open('ROI_accuracies.json', 'w') as json_file:
                json.dump(accuracies, json_file, indent=4)
    
    except Exception as e:
        print(f"There was an error for subject {subj}: ", e)



  0%|          | 0/30 [00:00<?, ?it/s]

====== Subject 05 ======
- Decoding FFC...
   Using top 256 nc values
   Accuracy: 53.018
   Std: 1.362
- Decoding V1...
   Using top 256 nc values
   Accuracy: 57.062
   Std: 5.011
- Decoding V2...
   Using top 256 nc values
   Accuracy: 56.673
   Std: 5.955
- Decoding V3...
   Using top 256 nc values
   Accuracy: 60.788
   Std: 3.843
- Decoding V3A...
   Using top 256 nc values
   Accuracy: 50.203
   Std: 1.882
- Decoding V3B...
   Using top 256 nc values
   Accuracy: 51.575
   Std: 5.895
- Decoding V3CD...
   Using top 256 nc values
   Accuracy: 52.362
   Std: 5.254
- Decoding V4...
   Using top 256 nc values
   Accuracy: 54.858
   Std: 5.326
- Decoding V6...
   Using top 256 nc values
   Accuracy: 49.82
   Std: 4.017
- Decoding V7...
   Using top 256 nc values
   Accuracy: 52.982
   Std: 2.758
- Decoding V8...
   Using top 256 nc values
   Accuracy: 51.23
   Std: 3.791
- Decoding VMV1...
   Using top 256 nc values
   Accuracy: 51.308
   Std: 4.938
- Decoding VMV2...
   Using top 25

e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]


- Decoding FFC...
   Using top 256 nc values


e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]
e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]
e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]
e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]
e:\Decoding\fmri-preprocessing\tc2see.py:76: RuntimeWarning: Mean of empty slice
  bold = (bold - np.nanmean(bold, axis=0)) / np.nanstd(bold, axis=0)
c:\Users\James\anaconda3\envs\representationsLab\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dty

...

   Accuracy: 48.802
   Std: 4.517
- Decoding 87...


e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]


   Using top 256 nc values


e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]
e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]
e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]
e:\Decoding\fmri-preprocessing\tc2see.py:70: RuntimeWarning: invalid value encountered in divide
  run_bold = (run_bold - predicted_bold) / group['bold_trend_std'][i]
e:\Decoding\fmri-preprocessing\tc2see.py:76: RuntimeWarning: Mean of empty slice
  bold = (bold - np.nanmean(bold, axis=0)) / np.nanstd(bold, axis=0)
c:\Users\James\anaconda3\envs\representationsLab\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dty

   Accuracy: 52.0
   Std: 2.946
====== Subject 09 ======
- Decoding FFC...
   Using top 256 nc values
   Accuracy: 52.433
   Std: 3.747
- Decoding V1...
   Using top 256 nc values
   Accuracy: 51.537
   Std: 3.639
- Decoding V2...
   Using top 256 nc values
   Accuracy: 53.397
   Std: 6.074
- Decoding V3...
   Using top 256 nc values
   Accuracy: 55.94
   Std: 2.49
- Decoding V3A...
   Using top 256 nc values
   Accuracy: 55.79
   Std: 3.521
- Decoding V3B...
   Using top 256 nc values
   Accuracy: 52.87
   Std: 4.56
- Decoding V3CD...
   Using top 256 nc values
   Accuracy: 59.19
   Std: 3.651
- Decoding V4...
   Using top 256 nc values
   Accuracy: 57.975
   Std: 3.402
- Decoding V6...
   Using top 256 nc values
   Accuracy: 49.313
   Std: 4.299
- Decoding V7...
   Using top 256 nc values
   Accuracy: 55.913
   Std: 5.27
- Decoding V8...
   Using top 256 nc values
   Accuracy: 54.315
   Std: 4.523
- Decoding VMV1...
   Using top 256 nc values
   Accuracy: 53.82
   Std: 2.808
- Decodi

In [8]:
print(accuracies)

{'05': {'FFC': 53.018, 'V1': 57.062, 'V2': 56.673, 'V3': 60.788, 'V3A': 50.203, 'V3B': 51.575, 'V3CD': 52.362, 'V4': 54.858, 'V6': 49.82, 'V7': 52.982, 'V8': 51.23, 'VMV1': 51.308, 'VMV2': 48.788, 'VMV3': 50.315, 'LO1': 56.142, 'LO2': 52.685, 'PIT': 57.13, 'VVC': 55.388, '140': 52.605, '11': 51.432, '85': 55.752, '83': 52.543, '82': 50.945, '87': 49.11}, '06': {'FFC': 52.107, 'V1': 51.945, 'V2': 56.733, 'V3': 50.385, 'V3A': 52.752, 'V3B': 53.277, 'V3CD': 52.135, 'V4': 50.412, 'V6': 53.123, 'V7': 48.278, 'V8': 50.038, 'VMV1': 51.598, 'VMV2': 48.247, 'VMV3': 57.537, 'LO1': 53.927, 'LO2': 48.358, 'PIT': 54.413, 'VVC': 52.842, '140': 46.828, '11': 52.482, '85': 51.143, '83': 53.087, '82': 49.222, '87': 51.813}, '07': {'FFC': 50.328, 'V1': 61.137, 'V2': 57.278, 'V3': 57.315, 'V3A': 50.66, 'V3B': 55.808, 'V3CD': 56.935, 'V4': 58.393, 'V6': 55.212, 'V7': 55.037, 'V8': 51.092, 'VMV1': 53.982, 'VMV2': 56.413, 'VMV3': 53.735, 'LO1': 54.703, 'LO2': 55.747, 'PIT': 54.717, 'VVC': 54.26, '140': 49.8